This is the part of our analysis where we work on Locality sensitive Hashing to do similarity search on the various block level data that we have taken from ACS data on BigQuery.

##Setup

In [0]:
!wget https://downloads.apache.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

--2020-05-16 20:28:51--  https://downloads.apache.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 2a01:4f8:10a:201a::2
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 232530699 (222M) [application/x-gzip]
Saving to: ‘spark-2.4.5-bin-hadoop2.7.tgz’

spark-2.4.5-bin-had 100%[===================>] 221.76M  26.3MB/s    in 9.1s    

2020-05-16 20:29:00 (24.3 MB/s) - ‘spark-2.4.5-bin-hadoop2.7.tgz’ saved [232530699/232530699]



In [0]:
!tar xf spark-2.4.5-bin-hadoop2.7.tgz

In [0]:
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate();

##Pre-Processing

In [0]:
import csv
import pandas as pd
import numpy as np

In [0]:
df_all = pd.read_csv('/content/drive/My Drive/blockgroup2010.csv')

In [0]:
columns_to_select_2 = ['geo_id','nonfamily_households','family_households','median_year_structure_built','rent_burden_not_computed','rent_over_50_percent','rent_40_to_50_percent','rent_35_to_40_percent','rent_30_to_35_percent','rent_25_to_30_percent','rent_20_to_25_percent','rent_15_to_20_percent','rent_10_to_15_percent','rent_under_10_percent','total_pop','male_pop','female_pop','median_age','white_pop','black_pop','asian_pop','hispanic_pop','amerindian_pop','other_race_pop','two_or_more_races_pop','not_hispanic_pop','commuters_by_public_transportation','households','median_income','income_per_capita','housing_units','vacant_housing_units','vacant_housing_units_for_rent','vacant_housing_units_for_sale','median_rent','percent_income_spent_on_rent','owner_occupied_housing_units','million_dollar_housing_units','mortgaged_housing_units','families_with_young_children','two_parent_families_with_young_children','two_parents_in_labor_force_families_with_young_children','two_parents_father_in_labor_force_families_with_young_children','two_parents_mother_in_labor_force_families_with_young_children','two_parents_not_in_labor_force_families_with_young_children','one_parent_families_with_young_children','father_one_parent_families_with_young_children','father_in_labor_force_one_parent_families_with_young_children','commute_10_14_mins','commute_15_19_mins','commute_20_24_mins','commute_25_29_mins','commute_30_34_mins','commute_45_59_mins','aggregate_travel_time_to_work','income_less_10000','income_10000_14999','income_15000_19999','income_20000_24999','income_25000_29999','income_30000_34999','income_35000_39999','income_40000_44999','income_45000_49999','income_50000_59999','income_60000_74999','income_75000_99999','income_100000_124999','income_125000_149999','income_150000_199999','income_200000_or_more','renter_occupied_housing_units_paying_cash_median_gross_rent','owner_occupied_housing_units_lower_value_quartile','owner_occupied_housing_units_median_value','owner_occupied_housing_units_upper_value_quartile','married_households','occupied_housing_units','housing_units_renter_occupied','dwellings_1_units_detached','dwellings_1_units_attached','dwellings_2_units','dwellings_3_to_4_units','dwellings_5_to_9_units','dwellings_10_to_19_units','dwellings_20_to_49_units','dwellings_50_or_more_units','mobile_homes','housing_built_2005_or_later','housing_built_2000_to_2004','housing_built_1939_or_earlier','male_under_5','male_5_to_9','male_10_to_14','male_15_to_17','male_18_to_19','male_20','male_21','male_22_to_24','male_25_to_29','male_30_to_34','male_35_to_39','male_40_to_44','male_45_to_49','male_50_to_54','male_55_to_59','male_60_61','male_62_64','male_65_to_66','male_67_to_69','male_70_to_74','male_75_to_79','male_80_to_84','male_85_and_over','female_under_5','female_5_to_9','female_10_to_14','female_15_to_17','female_18_to_19','female_20','female_21','female_22_to_24','female_25_to_29','female_30_to_34','female_35_to_39','female_40_to_44','female_45_to_49','female_50_to_54','female_55_to_59','female_60_to_61','female_62_to_64','female_65_to_66','female_67_to_69','female_70_to_74','female_75_to_79','female_80_to_84','female_85_and_over','white_including_hispanic','black_including_hispanic','amerindian_including_hispanic','asian_including_hispanic','commute_5_9_mins','commute_35_39_mins','commute_40_44_mins','commute_60_89_mins','commute_90_more_mins','households_retirement_income']

Here, we had chosen all columns which by any strech represented the demographic and income related features. In total there were 146 columns.






In [0]:
df_selected = df_all[columns_to_select_2]
df_selected = df_selected.dropna()
df_selected.to_csv('proj_data.csv', header = False, index= False)

In [0]:
df_selected['features'] = df_selected[df_selected.columns[1:]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
# df_selected['feats'] = [[float(idx) for idx in x.split(",")] for x in df_selected['features']]


In [0]:
df_selected.drop(df_selected.columns.difference(['geo_id','feats']), 1, inplace=True)

In [0]:
df_selected.to_csv('proj_data.csv', header = False, index= False)

In [0]:
lines = sc.textFile('proj_data.csv').mapPartitions(lambda part: csv.reader(part))

In [0]:
header = lines.first()
headerToIndex = dict([(header[i], i) for i in range(len(header))])

In [0]:
from pyspark.ml.feature import BucketedRandomProjectionLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col

In [0]:
data = lines.filter(lambda row: row != header)

##Analysis

In [0]:
df = spark.read.option("delimiter",",").csv("proj_data.csv")
dfUsed = df.select(col("_c0"), col("_c1"))
dfUsed.show()

+------------+---+
|         _c0|_c1|
+------------+---+
|490490102102| 28|
|120111103032| 55|
|550170111003|103|
|170978649031|125|
|360593015003| 48|
|371790210071| 56|
|510594602002| 55|
|360593020001| 27|
|250173001001|141|
|240054038021| 81|
|171978811081|179|
|260910623005| 69|
|420171050063| 76|
|420034742033| 66|
|360594115002| 24|
|190314502003|100|
|340373740002| 55|
|510594915021|138|
|420035638003| 41|
|261550313022|117|
+------------+---+
only showing top 20 rows



In [0]:
x = df.columns
cols = x[1:]

In [0]:
from pyspark.sql.types import IntegerType
for i in cols:
  df = df.withColumn(i, df[i].cast(IntegerType()))

In [0]:
df

DataFrame[_c0: string, _c1: int, _c2: int, _c3: int, _c4: int, _c5: int, _c6: int, _c7: int, _c8: int, _c9: int, _c10: int, _c11: int, _c12: int, _c13: int, _c14: int, _c15: int, _c16: int, _c17: int, _c18: int, _c19: int, _c20: int, _c21: int, _c22: int, _c23: int, _c24: int, _c25: int, _c26: int, _c27: int, _c28: int, _c29: int, _c30: int, _c31: int, _c32: int, _c33: int, _c34: int, _c35: int, _c36: int, _c37: int, _c38: int, _c39: int, _c40: int, _c41: int, _c42: int, _c43: int, _c44: int, _c45: int, _c46: int, _c47: int, _c48: int, _c49: int, _c50: int, _c51: int, _c52: int, _c53: int, _c54: int, _c55: int, _c56: int, _c57: int, _c58: int, _c59: int, _c60: int, _c61: int, _c62: int, _c63: int, _c64: int, _c65: int, _c66: int, _c67: int, _c68: int, _c69: int, _c70: int, _c71: int, _c72: int, _c73: int, _c74: int, _c75: int, _c76: int, _c77: int, _c78: int, _c79: int, _c80: int, _c81: int, _c82: int, _c83: int, _c84: int, _c85: int, _c86: int, _c87: int, _c88: int, _c89: int, _c90: i

In [0]:
import pyspark.sql.functions as f
columns = [df[i] for i in cols]

In [0]:
output = df.withColumn("features", f.array(columns)).select('_c0', 'features')

In [0]:
output.show()

+------------+--------------------+
|         _c0|            features|
+------------+--------------------+
|490490102102|[28, 301, 1993.0,...|
|120111103032|[55, 427, 1995.0,...|
|550170111003|[103, 314, 1978.0...|
|170978649031|[125, 622, 1966.0...|
|360593015003|[48, 278, 1943.0,...|
|371790210071|[56, 746, 2002.0,...|
|510594602002|[55, 578, 1976.0,...|
|360593020001|[27, 318, 1952.0,...|
|250173001001|[141, 378, 1958.0...|
|240054038021|[81, 602, 1968.0,...|
|171978811081|[179, 1078, 2000....|
|260910623005|[69, 466, 1991.0,...|
|420171050063|[76, 621, 1976.0,...|
|420034742033|[66, 724, 1973.0,...|
|360594115002|[24, 315, 1956.0,...|
|190314502003|[100, 159, 1975.0...|
|340373740002|[55, 501, 1978.0,...|
|510594915021|[138, 721, 1988.0...|
|420035638003|[41, 52, 1949.0, ...|
|261550313022|[117, 390, 1979.0...|
+------------+--------------------+
only showing top 20 rows



Assembling all the features vectors and using geo id as the key to determine different characteristics which had to be fed for LSH 

In [0]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(
    inputCols=[i for i in cols],
    outputCol="features")
final_df = assembler.transform(df)

In [0]:
final_df

DataFrame[_c0: string, _c1: int, _c2: int, _c3: int, _c4: int, _c5: int, _c6: int, _c7: int, _c8: int, _c9: int, _c10: int, _c11: int, _c12: int, _c13: int, _c14: int, _c15: int, _c16: int, _c17: int, _c18: int, _c19: int, _c20: int, _c21: int, _c22: int, _c23: int, _c24: int, _c25: int, _c26: int, _c27: int, _c28: int, _c29: int, _c30: int, _c31: int, _c32: int, _c33: int, _c34: int, _c35: int, _c36: int, _c37: int, _c38: int, _c39: int, _c40: int, _c41: int, _c42: int, _c43: int, _c44: int, _c45: int, _c46: int, _c47: int, _c48: int, _c49: int, _c50: int, _c51: int, _c52: int, _c53: int, _c54: int, _c55: int, _c56: int, _c57: int, _c58: int, _c59: int, _c60: int, _c61: int, _c62: int, _c63: int, _c64: int, _c65: int, _c66: int, _c67: int, _c68: int, _c69: int, _c70: int, _c71: int, _c72: int, _c73: int, _c74: int, _c75: int, _c76: int, _c77: int, _c78: int, _c79: int, _c80: int, _c81: int, _c82: int, _c83: int, _c84: int, _c85: int, _c86: int, _c87: int, _c88: int, _c89: int, _c90: i

In [0]:
brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", bucketLength=2.0,
                                      numHashTables=10)

In [0]:
model = brp.fit(final_df)

In [0]:
model.transform(final_df).select(col("_c0"), col("features"), col("hashes")).show()

+------------+--------------------+--------------------+
|         _c0|            features|              hashes|
+------------+--------------------+--------------------+
|490490102102|[28.0,301.0,1993....|[[814.0], [5234.0...|
|120111103032|[55.0,427.0,1995....|[[1142.0], [7406....|
|550170111003|[103.0,314.0,1978...|[[-277.0], [2822....|
|170978649031|[125.0,622.0,1966...|[[885.0], [11512....|
|360593015003|[48.0,278.0,1943....|[[6757.0], [21529...|
|371790210071|[56.0,746.0,2002....|[[91.0], [10845.0...|
|510594602002|[55.0,578.0,1976....|[[2999.0], [15216...|
|360593020001|[27.0,318.0,1952....|[[9104.0], [28043...|
|250173001001|[141.0,378.0,1958...|[[232.0], [4960.0...|
|240054038021|[81.0,602.0,1968....|[[1945.0], [19092...|
|171978811081|[179.0,1078.0,200...|[[-496.0], [2308....|
|260910623005|[69.0,466.0,1991....|[[-408.0], [3754....|
|420171050063|[76.0,621.0,1976....|[[1671.0], [9442....|
|420034742033|[66.0,724.0,1973....|[[-1165.0], [4551...|
|360594115002|[24.0,315.0,1956.

*After fitting our LSH module to the feature vectors, we find out the Hashes generated using the features*

In [0]:
threshold = 50000      # euclidean distance
similar = model.approxSimilarityJoin(final_df, final_df, threshold).filter("distCol != 0")

In [214]:
# spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
similar.show()

+--------------------+--------------------+------------------+
|            datasetA|            datasetB|           distCol|
+--------------------+--------------------+------------------+
|[420990302024, 28...|[370270308004, 30...| 21316.86445047676|
|[482090109062, 19...|[120570128001, 18...|25100.034302765405|
|[471630411002, 25...|[470039507001, 78...| 30704.55728389517|
|[250158201012, 34...|[550219706002, 21...|35918.749811205846|
|[420032904004, 10...|[181650203001, 89...|25065.395189384108|
|[470539673001, 86...|[292074703002, 13...|28985.615294487023|
|[481410102172, 18...|[340076071003, 95...| 35824.71212724535|
|[511210210002, 29...|[100050507011, 34...|24134.784461436568|
|[290179502003, 12...|[130939701001, 18...|21841.441893794465|
|[10950302012, 160...|[51250105112, 349...|48126.181720556226|
|[100050503012, 24...|[410099708003, 37...| 38643.12337272959|
|[481210215211, 11...|[420912088021, 18...|16162.228930441494|
|[180179515001, 21...|[370850705003, 18...| 34865.40996

We run our code for a single hash table. Increasing the number of hash tables will increase the accuracy, but also the program’s communication cost and running time.

In [0]:
final_df.select(col('features')).take(1)    # random feature taken for finding nearest neighbor

[Row(features=DenseVector([28.0, 301.0, 1993.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1431.0, 783.0, 648.0, 19.0, 1431.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1431.0, 35.0, 329.0, 102539.0, 23440.0, 363.0, 34.0, 24.0, 0.0, 2001.0, 10.0, 327.0, 0.0, 214.0, 97.0, 94.0, 73.0, 21.0, 0.0, 0.0, 3.0, 0.0, 0.0, 35.0, 65.0, 87.0, 65.0, 64.0, 83.0, 18205.0, 0.0, 10.0, 0.0, 10.0, 0.0, 16.0, 21.0, 0.0, 0.0, 31.0, 30.0, 46.0, 78.0, 34.0, 35.0, 18.0, 2001.0, 278000.0, 337100.0, 392700.0, 241.0, 329.0, 2.0, 363.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 59.0, 0.0, 28.0, 120.0, 104.0, 80.0, 77.0, 22.0, 0.0, 0.0, 19.0, 28.0, 24.0, 59.0, 89.0, 47.0, 22.0, 0.0, 9.0, 11.0, 38.0, 6.0, 0.0, 0.0, 0.0, 59.0, 65.0, 98.0, 101.0, 0.0, 7.0, 0.0, 12.0, 16.0, 30.0, 43.0, 59.0, 50.0, 37.0, 17.0, 7.0, 14.0, 17.0, 0.0, 6.0, 10.0, 0.0, 0.0, 1431.0, 0.0, 0.0, 0.0, 18.0, 8.0, 13.0, 77.0, 14.0, 62.0]))]

In [0]:
key = Vectors.dense([28.0, 301.0, 1993.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1431.0, 783.0, 648.0, 19.0, 1431.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1431.0, 35.0, 329.0, 102539.0, 23440.0, 363.0, 34.0, 24.0, 0.0, 2001.0, 10.0, 327.0, 0.0, 214.0, 97.0, 94.0, 73.0, 21.0, 0.0, 0.0, 3.0, 0.0, 0.0, 35.0, 65.0, 87.0, 65.0, 64.0, 83.0, 18205.0, 0.0, 10.0, 0.0, 10.0, 0.0, 16.0, 21.0, 0.0, 0.0, 31.0, 30.0, 46.0, 78.0, 34.0, 35.0, 18.0, 2001.0, 278000.0, 337100.0, 392700.0, 241.0, 329.0, 2.0, 363.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 59.0, 0.0, 28.0, 120.0, 104.0, 80.0, 77.0, 22.0, 0.0, 0.0, 19.0, 28.0, 24.0, 59.0, 89.0, 47.0, 22.0, 0.0, 9.0, 11.0, 38.0, 6.0, 0.0, 0.0, 0.0, 59.0, 65.0, 98.0, 101.0, 0.0, 7.0, 0.0, 12.0, 16.0, 30.0, 43.0, 59.0, 50.0, 37.0, 17.0, 7.0, 14.0, 17.0, 0.0, 6.0, 10.0, 0.0, 0.0, 1431.0, 0.0, 0.0, 0.0, 18.0, 8.0, 13.0, 77.0, 14.0, 62.0])
model.approxNearestNeighbors(final_df, key, 10).show()

+------------+----+----+----+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+------+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+------+------+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------------------+--------------------+------------------+
|         _c0| _c1| _c2| _c3|_c4|_c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|_c25|_c26|_c27|  _c28

For an approximate similarity join and the approximate nearest neighbor command, the number of hash tables can be used to trade off between running time and false positive rate.

In [0]:
dem_cols = ['geo_id','total_pop','male_pop','female_pop','median_age','white_pop','black_pop','asian_pop','hispanic_pop','amerindian_pop','other_race_pop', 'median_income','income_per_capita', 'median_rent']

In [0]:
df_dem = df_all[dem_cols]
df_dem = df_dem.dropna()

We found out that due to very high dimensionality of our data, the hash functions that were created were not very optimal. We then tried to use only the most critical demographic features to see if the results were any better.

In [0]:
df_dem.to_csv('dem_data.csv', header = False, index= False)

In [223]:
df = spark.read.option("delimiter",",").csv("dem_data.csv")
dfUsed = df.select(col("_c0"), col("_c1"))
dfUsed.show()

+------------+----+
|         _c0| _c1|
+------------+----+
|510594916021|2920|
|490490102102|1431|
|511076107023|1169|
|360593031022| 963|
|120111103032|1418|
|511076118052|2177|
|360179706012| 780|
|390170111163|3117|
|120990078223|2332|
|171670034005| 830|
|371790210072|2159|
|360595177012|2385|
| 60590218242|1491|
|550170111003|1160|
|170978649031|2317|
|360593015003|1053|
|481390602112|1314|
| 60590626313| 601|
|371790210071|2768|
|340030321021|1833|
+------------+----+
only showing top 20 rows



In [0]:
x = df.columns
cols = x[1:]

from pyspark.sql.types import IntegerType
for i in cols:
  df = df.withColumn(i, df[i].cast(IntegerType()))

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=[i for i in cols],
    outputCol="features")
final_df = assembler.transform(df)

In [0]:
brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", bucketLength=2.0,
                                      numHashTables=10)

In [0]:
model = brp.fit(final_df)

In [228]:
model.transform(final_df).select(col("_c0"), col("features"), col("hashes")).show()

+------------+--------------------+--------------------+
|         _c0|            features|              hashes|
+------------+--------------------+--------------------+
|510594916021|[2920.0,1397.0,15...|[[21857.0], [1894...|
|490490102102|[1431.0,783.0,648...|[[22676.0], [1792...|
|511076107023|[1169.0,606.0,563...|[[29269.0], [2374...|
|360593031022|[963.0,406.0,557....|[[23418.0], [2058...|
|120111103032|[1418.0,700.0,718...|[[24039.0], [2026...|
|511076118052|[2177.0,990.0,118...|[[31622.0], [2555...|
|360179706012|[780.0,419.0,361....|[[12302.0], [1024...|
|390170111163|[3117.0,1623.0,14...|[[28104.0], [2334...|
|120990078223|[2332.0,1139.0,11...|[[22607.0], [1858...|
|171670034005|[830.0,468.0,362....|[[16100.0], [1307...|
|371790210072|[2159.0,1154.0,10...|[[33801.0], [2823...|
|360595177012|[2385.0,1135.0,12...|[[48456.0], [4043...|
| 60590218242|[1491.0,808.0,683...|[[37028.0], [3161...|
|550170111003|[1160.0,596.0,564...|[[11093.0], [9501...|
|170978649031|[2317.0,1158.0,11

In [0]:
threshold = 100      # euclidean distance
similar = model.approxSimilarityJoin(final_df, final_df, threshold).filter("distCol != 0")

In [233]:
similar.show()

+--------------------+--------------------+------------------+
|            datasetA|            datasetB|           distCol|
+--------------------+--------------------+------------------+
|[421279603001, 10...|[360099611004, 10...| 75.07329751649384|
|[201079552002, 11...|[261590101005, 11...| 93.03762679690406|
|[380779714002, 79...|[420391119001, 81...|58.532042506647585|
|[460619641003, 11...|[180950101001, 11...| 85.17628778010932|
|[295101083002, 89...|[245102007015, 87...|  94.2178327069775|
|[551239607004, 11...|[421150326001, 11...|              79.0|
|[360010127003, 58...|[172010015002, 56...|  98.2496819333274|
|[550679603002, 64...|[270935605003, 64...|  77.9743547584717|
|[120860006043, 71...|[481410023003, 73...| 72.39475119095306|
|[261590101005, 11...|[201079552002, 11...| 93.03762679690406|
|[391517113111, 16...|[390610204014, 16...| 92.50405396521819|
|[550870131003, 76...|[410099703005, 73...| 90.69178573608527|
|[470619551001, 96...|[180459580001, 97...| 88.07383266

And indeed, our hunch was correct, the dimensionality was playing a key issue in the formation of very irregular hashes. The distance got reduced by a factor of 100. This information would be our initial analysis which we would be later on using for Clustering and Analyzing various demographic features and their dependence on income levels.

After we were already on our course in our analysis, we came across this paper [NIPS 2015](http://papers.nips.cc/paper/5893-practical-and-optimal-lsh-for-angular-distance). We realized that for distributed aspect of Locality Sensitive Hashing, a multi-probe based method would be faster and more efficient. We weren't able to implement it ourselves, but found a library in Scala which works well. We tried to validate our results using FALCONN library as well.

In [234]:
!pip install FALCONN

     |████████████████████████████████| 1.4MB 3.5MB/s 
  Created wheel for FALCONN: filename=FALCONN-1.3.1-cp36-cp36m-linux_x86_64.whl size=10581603 sha256=7a045214d3861023bd12d17fa3eb07c195e8e7e865c03882e38275a8f1905645
  Stored in directory: /root/.cache/pip/wheels/bf/36/96/d5538901888620fc0343c1ed9d5f87fce00869e00c12056ef8
Successfully built FALCONN


In [0]:
dataset = df_selected.to_numpy()
dataset = dataset.astype('float32')     # converting data to numpy
dataset /= np.linalg.norm(dataset, axis=1).reshape(-1, 1)   # standardizing the data

In [0]:
number_of_queries = 1000
number_of_tables = 50
np.random.seed(4057218)
np.random.shuffle(dataset)
queries = dataset[len(dataset) - number_of_queries:]
dataset = dataset[:len(dataset) - number_of_queries]

In [0]:
answers = []
for query in queries:
    answers.append(np.dot(dataset, query).argmax())   # computing the dot product for LSH

In [0]:
center = np.mean(dataset, axis=0)   # mean centering the data
dataset -= center
queries -= center

In [0]:
# setting up all parameters as required

import falconn

params_cp = falconn.LSHConstructionParameters()
params_cp.dimension = len(dataset[0])
params_cp.lsh_family = falconn.LSHFamily.CrossPolytope
params_cp.distance_function = falconn.DistanceFunction.EuclideanSquared
params_cp.l = number_of_tables
params_cp.num_rotations = 1
params_cp.seed = 5721840
params_cp.num_setup_threads = 0
params_cp.storage_hash_table = falconn.StorageHashTable.BitPackedFlatHashTable
falconn.compute_number_of_hash_functions(14, params_cp)
table = falconn.LSHIndex(params_cp)
table.setup(dataset)
query_object = table.construct_query_object()
number_of_probes = number_of_tables

In [0]:
# counting the number of probes for analysis
def evaluate_number_of_probes(number_of_probes):
    query_object.set_num_probes(number_of_probes)
    score = 0
    for (i, query) in enumerate(queries):
        if answers[i] in query_object.get_candidates_with_duplicates(
                query):
            score += 1
    return float(score) / len(queries)

In [0]:
# running binary search for the probes
while True:
    accuracy = evaluate_number_of_probes(number_of_probes)
    print('{} -> {}'.format(number_of_probes, accuracy))
    if accuracy >= 0.9:
        break
    number_of_probes = number_of_probes * 2
if number_of_probes > number_of_tables:
    left = number_of_probes // 2
    right = number_of_probes
    while right - left > 1:
        number_of_probes = (left + right) // 2
        accuracy = evaluate_number_of_probes(number_of_probes)
        print('{} -> {}'.format(number_of_probes, accuracy))
        if accuracy >= 0.9:
            right = number_of_probes
        else:
            left = number_of_probes
    number_of_probes = right
print('Done')
print('{} probes'.format(number_of_probes))

In [0]:
score = 0
for (i, query) in enumerate(queries):
    if query_object.find_nearest_neighbor(query) == answers[i]:
        score += 1
print('Precision: {}'.format(float(score) / len(queries)))
query_object.find_near_neighbors(query, 100)[:20]

Although we see some differences from our analysis, the most notable change was the change in the speed. We look forward to exploring this more.